In [1]:
from __future__ import print_function

import pandas as pd 
import numpy as np 
import sklearn

# NLTK/NLP
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
from nltk import FreqDist, word_tokenize
import string, re
import urllib
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.collocations import *
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import KeyedVectors

# Classifiers 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split


#Sampling
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

import sklearn.decomposition as decomposition

#Visualization
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

import warnings 
warnings.filterwarnings("ignore")

import csv

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

from sklearn.metrics import average_precision_score, auc, roc_curve, precision_recall_curve



Using TensorFlow backend.


In [2]:
import pickle

In [3]:
%run custom_functions_test.ipynb

In [4]:
with open('data/cleaned-reshuffled.pkl', 'rb') as f:
	df = pickle.load(f)

In [5]:
df.head()

,id,label,tweet,tidy_tweet,no_hash_tweet,tokenized_tweet,stemmed_tokens,lemmatized_tokens,lem_tweet,stem_tweet
0,29727,0,sad to see the scenes of hooligans pre #engrus...,sad to see the scenes of hooligans pre #engrus...,sad to see the scenes of hooligans pre engrus ...,"[sad, to, see, the, scenes, of, hooligans, pre...","[sad, to, see, the, scene, of, hooligan, pre, ...","[sad, to, see, the, scene, of, hooligan, pre, ...",sad to see the scenes of hooligans pre engrus ...,sad to see the scenes of hooligans pre engrus ...
1,14466,0,#gooddyeyoung #yoyoyo !! super happy to be ap...,#gooddyeyoung #yoyoyo super happy to be apa of...,gooddyeyoung yoyoyo super happy to be apa of t...,"[gooddyeyoung, yoyoyo, super, happy, to, be, a...","[gooddyeyoung, yoyoyo, super, happi, to, be, a...","[gooddyeyoung, yoyoyo, super, happy, to, be, a...",gooddyeyoung yoyoyo super happy to be apa of t...,gooddyeyoung yoyoyo super happy to be apa of t...
2,18194,0,queen evil's bihdayð#lnic #lnicjustanevilbd...,queen evil s bihday #lnic #lnicjustanevilbday ...,queen evil s bihday lnic lnicjustanevilbday bi...,"[queen, evil, s, bihday, lnic, lnicjustanevilb...","[queen, evil, s, bihday, lnic, lnicjustanevilb...","[queen, evil, s, bihday, lnic, lnicjustanevilb...",queen evil s bihday lnic lnicjustanevilbday bi...,queen evil s bihday lnic lnicjustanevilbday bi...
3,18283,1,@user you might be a libtard if... #libtard #...,you might be a libtard if #libtard #sjw #liber...,you might be a libtard if libtard sjw liberal ...,"[you, might, be, a, libtard, if, libtard, sjw,...","[you, might, be, a, libtard, if, libtard, sjw,...","[you, might, be, a, libtard, if, libtard, sjw,...",you might be a libtard if libtard sjw liberal ...,you might be a libtard if libtard sjw liberal ...
4,25845,0,what are your goals? find out here... #smile...,what are your goals find out here #smile,what are your goals find out here smile,"[what, are, your, goals, find, out, here, smile]","[what, are, your, goal, find, out, here, smile]","[what, are, your, goal, find, out, here, smile]",what are your goals find out here smile,what are your goals find out here smil


## Train / Val / Test Split

In [7]:
X = df.drop(['label'], axis = 1)

In [8]:
y = df['label']

In [9]:
#splitting into train and test 
X_model, X_test, y_model, y_test = train_test_split(X, y, stratify = y,  test_size=0.20, random_state=123)

#splitting "model" into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size=0.20, random_state=123)

# df_train_full = X_train.copy()
# df_train_full['label']= y_train
# train_full_df.to_csv('train_full_df.csv')

In [10]:
y.value_counts(normalize=True)

0    0.929854
1    0.070146
Name: label, dtype: float64

## Comparing Vectorization and Method Performance

In [18]:
count_vect = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer(min_df=.001)
tfidf_ngram = TfidfVectorizer(ngram_range=(1,2), min_df=.001)
tfidf_ngram2 = TfidfVectorizer(ngram_range=(2,3),min_df=.001)

logreg = LogisticRegression(random_state=10)
rfc = RandomForestClassifier(random_state=10)
nb = GaussianNB()
svc = SVC(random_state=10)

vectorization_list = [('COUNT_VECTORIZER', count_vect),
                      ('TFIDF_VECTORIZER', tfidf_vectorizer),
                      ('TFIDF_NGRAM_1_2', tfidf_ngram),
                      ('TFIDF_NGRAM_2_3', tfidf_ngram2)]



## Testing Fun

In [ ]:
%run custom_functions_test.ipynb

In [ ]:
X_train_up2, y_train_up2, X_train_transf2, X_val_transf2 = \
upsample_training_data3(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, log, count_vect)

In [ ]:
X_train.lem_tweet.shape

In [ ]:
X_train_up2.shape

In [ ]:
X_train_transf2.shape

In [ ]:
X_val_transf2.shape

In [ ]:
y_train_up.shape

In [ ]:
X_train_transf.shape

In [ ]:
X_val.lem_tweet.shape

In [ ]:
X_val_transf.shape

In [11]:
%run custom_functions_test.ipynb

In [ ]:
X_up, y_up, X_train_transformed, X_val_transformed = \
upsample_training_data3(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, LogisticRegression(), tfidf_vectorizer)

In [ ]:
X_train_transformed.shape

In [ ]:
X_up.lem_tweet

In [ ]:
X_up.values.ravel()

In [ ]:
test = pd.DataFrame(X_train_up.lem_tweet)

In [ ]:
vect = count_vect.fit_transform(X_up)
vect.shape

In [ ]:
X_train_transformed.shape

In [ ]:
vect = count_vect.fit_transform(test)
vect.shape

In [ ]:
%run custom_functions_test.ipynb

In [ ]:
X_train_transformed, X_val_transformed, y_train_pred, y_val_pred, y_val_pred_prob, matrix, \
compare_metrics, compare_predictions = \
single_vector_model3(X_train_up.lem_tweet, y_train_up, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [ ]:
X_train_transformed, X_val_transformed, y_train_pred, y_val_pred, y_val_pred_prob, matrix, \
compare_metrics, compare_predictions = \
upsample_vector_model(X_train_up.lem_tweet, y_train_up, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [ ]:
compare_metrics

In [ ]:
compare_predictions.head()

In [ ]:
%run custom_functions_test.ipynb

In [12]:
X_train_transformed, X_val_transformed, y_train_pred, y_val_pred2, y_val_pred_prob, \
compare_predictions, metrics_dict2 = \
smote_vector_model2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [13]:
X_train_transformed.shape

(20455, 28610)

In [14]:
X_train_transformed, X_val_transformed, y_train_pred, y_val_pred2, y_val_pred_prob, \
compare_predictions, metrics_dict2 = \
upsample_vector_model2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [15]:
X_train_transformed, X_val_transformed, y_train_pred, y_val_pred2, y_val_pred_prob, \
compare_predictions, metrics_dict2 = \
downsample_vector_model2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [ ]:
y_val_pred2

In [ ]:
confusion_matrix (y_val, y_val_pred2)

In [22]:
%run custom_functions_test.ipynb

In [ ]:
X_train_down_transformed, X_val_transformed, y_train_down, y_val_pred2, yval_pp, \
 compare_predictions, metrics_dict = \
upsample_vector_model2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [27]:
X_train_down_transformed, X_val_transformed, y_train_down, y_val_pred2, yval_pp, \
 compare_predictions, metrics_dict = \
single_vector_model2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, LogisticRegression(), count_vect)

In [30]:
%run custom_functions_test.ipynb

In [31]:
X_train_t, X_val_t, y_train_pred, y_val_pred3, y_val_prob, metrics2, pred_df = \
wrapper_single_vectorization(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                              LogisticRegression(), count_vect, sampling='smote')

In [32]:
X_train_t, X_val_t, y_train_pred, y_val_pred3, y_val_prob, metrics3, pred_df = \
wrapper_single_vectorization(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                              LogisticRegression(), count_vect, sampling='upsample')

In [33]:
X_train_t, X_val_t, y_train_pred, y_val_pred3, y_val_prob, metrics3, pred_df = \
wrapper_single_vectorization(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                              LogisticRegression(), count_vect, sampling='downsample')

In [34]:
X_train_t, X_val_t, y_train_pred, y_val_pred3, y_val_prob, metrics3, pred_df = \
wrapper_single_vectorization(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, log, count_vect)

In [92]:
%run custom_functions_test.ipynb

In [100]:
metrics_results = downsample_compare_vectorization_model(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, 
                                                         LogisticRegression(penalty = 'l1', random_state=1),
                                                         vectorization_list)

In [101]:
metrics_results

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.93,0.85,0.84,0.72
Train F1,0.93,0.85,0.84,0.64
Train Precision,0.94,0.87,0.87,0.90
Train Recall,0.91,0.82,0.81,0.50
Validation Accuracy,0.85,0.83,0.82,0.86
Validation F1,0.40,0.35,0.35,0.27
Validation Precision,0.27,0.23,0.23,0.21
Validation Recall,0.76,0.73,0.74,0.38


In [106]:
metrics_results = \
wrapper_compare_vectorizations2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                             LogisticRegression(penalty='l1', random_state =1), 
                                vectorization_list, sampling = 'upsample')
pd.DataFrame(metrics_results)

In [108]:
metrics_results = \
wrapper_compare_vectorizations2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                             LogisticRegression(penalty='l1', random_state =1), 
                                vectorization_list, sampling = 'downsample')
pd.DataFrame(metrics_results)

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.93,0.85,0.84,0.72
Train F1,0.93,0.85,0.84,0.64
Train Precision,0.94,0.87,0.87,0.90
Train Recall,0.91,0.82,0.81,0.50
Validation Accuracy,0.85,0.83,0.82,0.86
Validation F1,0.40,0.35,0.35,0.27
Validation Precision,0.27,0.23,0.23,0.21
Validation Recall,0.76,0.73,0.74,0.38


In [109]:
metrics_results = \
wrapper_compare_vectorizations2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                             LogisticRegression(penalty='l1', random_state =1), 
                                vectorization_list)
pd.DataFrame(metrics_results)

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.97,0.95,0.95,0.93
Train F1,0.79,0.53,0.52,0.14
Train Precision,0.96,0.84,0.84,0.91
Train Recall,0.67,0.39,0.38,0.08
Validation Accuracy,0.96,0.95,0.95,0.94
Validation F1,0.62,0.50,0.48,0.13
Validation Precision,0.81,0.78,0.79,0.96
Validation Recall,0.50,0.37,0.35,0.07


In [110]:
metrics_results = \
wrapper_compare_vectorizations2(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val,
                             LogisticRegression(penalty='l1', random_state =1), 
                                vectorization_list, sampling = 'gibberish')
pd.DataFrame(metrics_results)

Sampling method does not exist. Indicate: smote, upsample or downsample


""


## Naive Bayes

In [ ]:
np.random.seed(0)

In [ ]:
NB_compare_vectorization_model(X_train.lem_tweet, y_train, 
                                   X_val.lem_tweet, y_val, GaussianNB())

## Logistic Regression

In [ ]:
%run custom_functions_test.ipynb

In [24]:
#Logistic Regression: compare vectorizers with no presets
lr_results1 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(solver = 'lbfgs', random_state = 10), 
                                            vectorization_list)

lr_results1

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.95,0.95,0.93
Train F1,0.89,0.48,0.48,0.14
Train Precision,0.99,0.88,0.89,0.93
Train Recall,0.81,0.33,0.33,0.08
Validation Accuracy,0.96,0.95,0.95,0.94
Validation F1,0.62,0.44,0.43,0.13
Validation Precision,0.86,0.84,0.88,0.96
Validation Recall,0.49,0.30,0.29,0.07


In [25]:
#Logistic Regression: compare vectorizers using lemmitizing + class balances
lr_results2 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(class_weight= 'balanced', 
                            solver = 'lbfgs', random_state = 10), vectorization_list)
lr_results2

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.89,0.90,0.54
Train F1,0.93,0.55,0.58,0.23
Train Precision,0.86,0.39,0.41,0.13
Train Recall,0.99,0.92,0.94,0.92
Validation Accuracy,0.95,0.87,0.88,0.52
Validation F1,0.66,0.43,0.44,0.18
Validation Precision,0.62,0.30,0.31,0.10
Validation Recall,0.71,0.75,0.74,0.83


In [26]:
#Logistic Regression: compare vectorizers using lemmitizing + upsampling
lr_results3 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(solver = 'lbfgs', 
                            random_state = 10), vectorization_list, sampling = 'upsample')
lr_results3

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.94,0.96,0.80
Train F1,0.99,0.94,0.96,0.77
Train Precision,0.99,0.93,0.94,0.88
Train Recall,1.00,0.96,0.98,0.69
Validation Accuracy,0.95,0.91,0.91,0.87
Validation F1,0.65,0.51,0.53,0.32
Validation Precision,0.62,0.38,0.41,0.24
Validation Recall,0.68,0.76,0.76,0.47


In [27]:
#Logistic Regression: compare vectorizers using lemmitizing + downsampling
lr_results4 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(random_state = 10, solver = 'lbfgs'), 
                            vectorization_list, sampling = 'downsample')
lr_results4

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.92,0.93,0.81
Train F1,0.99,0.92,0.93,0.79
Train Precision,0.99,0.91,0.92,0.90
Train Recall,0.99,0.92,0.93,0.71
Validation Accuracy,0.86,0.84,0.84,0.82
Validation F1,0.42,0.40,0.40,0.29
Validation Precision,0.29,0.26,0.27,0.19
Validation Recall,0.82,0.82,0.81,0.54


In [50]:
#Logistic Regression: compare vectorizers using lemmitizing + smote
lr_results5 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(random_state = 10, solver = 'lbfgs'), 
                            vectorization_list, sampling = 'smote')
lr_results5

AttributeError: 'NoneType' object has no attribute 'items'

In [41]:
%run custom_functions_test.ipynb

In [43]:
#Logistic Regression: compare vectorizers using lemmitizing + smote
lr_results7 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(random_state = 10, solver = 'lbfgs'), 
                            vectorization_list, sampling = 'smote', sample_class= 'all')
lr_results7

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.96,0.90,0.90,0.57
Train F1,0.77,0.54,0.56,0.22
Train Precision,0.68,0.40,0.42,0.13
Train Recall,0.88,0.84,0.85,0.87
Validation Accuracy,0.89,0.88,0.89,0.55
Validation F1,0.45,0.44,0.44,0.18
Validation Precision,0.34,0.31,0.32,0.10
Validation Recall,0.68,0.72,0.71,0.78


In [ ]:
#Logistic Regression: compare vectorizers using lemmitizing + smote
lr_results7 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            LogisticRegression(random_state = 10, solver = 'lbfgs'), 
                            vectorization_list, sampling = 'smote', sample_class='not majority')
lr_results7

In [ ]:
with open('mycsvfile.csv','a') as f:
    w = csv.writer(f)
    w.writerows(LR_cw_lemm.items())

In [ ]:
(pd.DataFrame.from_dict(data= LR_cw_lemm).to_csv('dict_file.csv', header=False))

### Grid Searching

##  Final Logistic Regression Model

In [ ]:
metrics_dict = wrapper_compare_vectorizations(X_train_up.lem_tweet, y_train_up, X_val.lem_tweet, y_val, 
                            LogisticRegression(class_weight='balanced', penalty = 'l1'), 
                            vectorization_list, sampling='help')
pd.DataFrame(metrics_dict)

In [ ]:
weighted_predictions_val = []
for item in logreg.predict_proba(X_val_transformed):
    if item[0] <= .85:
        weighted_predictions_val.append(1)
    else:
        weighted_predictions_val.append(0)

In [ ]:
#with threshhold adjustment
pd.DataFrame(confusion_matrix(y_val, weighted_predictions_val), index = ['actual 0', 'actual 1'], columns = ['predicted 0', 'predicted 1'])

In [ ]:
pred_df[pred_df['actual_class'] != pred_df['predicted_class']]
pred_df.head()

In [ ]:
pred_df['weighted_predictions'] = weighted_predictions_val
pred_df[pred_df['actual_class'] != pred_df['weighted_predictions']].head()

In [ ]:
df.tweet[11418]

## Testing on Test Set

In [ ]:
X_train_transformed, metrics_dict, train_confusion_matrix, y_test_pred, y_test_prob, test_df = \
\
wrapper_single_vectorization(X_train_up.lem_tweet, y_train_up, X_test.lem_tweet, y_test, 
                            logreg, count_vect, apply_smote = False)

## SVM


In [ ]:
#class weight = balanced + lemmatized
svm_metrics_balance, svm_X_train_transformed, svm_X_val_transformed = \
wrapper_compare_vectorizations(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, 
                                   SVC(class_weight ='balanced', gamma='auto', random_state = 10), vectorization_list, apply_smote=False)

In [ ]:
pd.DataFrame(svm_metrics_balance)

In [ ]:
#SMOTE + lemmatized 
svm_metrics_smote, svm_X_train_smote, svm_X_val_smote   = \
wrapper_compare_vectorizations(X_train.lem_tweet, y_train, X_val.lem_tweet, y_val, 
                                   SVC(class_weight ='balanced', gamma='auto', random_state = 10), vectorization_list, apply_smote=False)

In [ ]:
pd.DataFrame(svm_metrics_smote)

In [ ]:
#upsampling + lemmatized
svm_metrics_up, svm_X_train_up, svm_X_val_up = \
wrapper_compare_vectorizations(X_train_up.lem_tweet, y_train, X_val_up.lem_tweet, y_val, 
                                   SVC(gamma='auto', random_state = 10), vectorization_list, apply_smote=False)
pd.DataFrame(svm_metrics_up)

#### Grid Searching:

In [ ]:
X_train_tfid2 =  tfidf_ngram2.fit_transform(X_train_up.lemmatized_tweet)
X_val_tfid2 =  tfidf_ngram2.transform(X_val.lemmatized_tweet)

In [ ]:
# svc = SVC(kernel='linear', C=1, gamma=1, class_weight ='balanced')

params = {
'C': [0.1,.2, .3, 0.8,1,1.2,1.4],
'kernel':['linear', 'rbf'],
'gamma' :[0.1,0.8,1,1.2,1.4]
}

svm_gs= GridSearchCV(svc, param_grid = params, cv = 3)

scores = ['f1','accuracy','recall']

In [ ]:
svm_gs.fit(X_train_tfid2, y_train_up)

In [ ]:
svm_gs.best_estimator_

In [ ]:
single_vector_model()

In [ ]:
compare_vectorization_model(X_train_up.lemmatized_tweet, y_train_up, X_val.lemmatized_tweet, y_val, 
                                   SVC(C=1.2, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.4, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))

# Random Forest

### Compare Multiple Methods:

In [ ]:
  RandomForestClassifier(max_depth= 20, 
                                   n_estimators = 100, class_weight='balanced', random_state=10)

In [33]:
#Random Forest: compare vectorizers with lemmatizing; no hyperparameter tuning
rfc_results1 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            RandomForestClassifier(random_state=10), vectorization_list)
rfc_results1

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.99,0.99,0.95
Train F1,0.96,0.94,0.94,0.43
Train Precision,1.00,1.00,1.00,0.93
Train Recall,0.92,0.88,0.89,0.28
Validation Accuracy,0.96,0.96,0.96,0.94
Validation F1,0.59,0.55,0.57,0.17
Validation Precision,0.92,0.83,0.84,0.58
Validation Recall,0.43,0.41,0.43,0.10


In [34]:
#Random Forest: compare vectorizers with lemmatizing and class weights balanced
rfc_results2 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            RandomForestClassifier(random_state=10, class_weight = 'balanced'), 
                            vectorization_list)
rfc_results2

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.99,0.99,0.62
Train F1,0.95,0.94,0.93,0.25
Train Precision,1.00,0.97,0.97,0.15
Train Recall,0.91,0.90,0.90,0.92
Validation Accuracy,0.96,0.95,0.95,0.58
Validation F1,0.50,0.52,0.51,0.18
Validation Precision,0.96,0.80,0.77,0.10
Validation Recall,0.34,0.39,0.38,0.71


In [35]:
#Random Forest: compare vectorizers with lemmatizing and upsampling
rfc_results3 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            RandomForestClassifier(random_state=10), 
                            vectorization_list, sampling = 'upsample')
rfc_results3

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,1.00,1.00,1.00,0.85
Train F1,1.00,1.00,1.00,0.83
Train Precision,1.00,1.00,1.00,0.94
Train Recall,1.00,1.00,1.00,0.74
Validation Accuracy,0.96,0.95,0.96,0.91
Validation F1,0.57,0.57,0.58,0.36
Validation Precision,0.86,0.73,0.78,0.32
Validation Recall,0.42,0.47,0.47,0.40


In [36]:
#Random Forest: compare vectorizers with lemmatizing and downsampling
rfc_results4 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            RandomForestClassifier(random_state=10), 
                            vectorization_list, sampling = 'downsample')
rfc_results4

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,0.99,0.99,0.86
Train F1,0.99,0.99,0.99,0.85
Train Precision,1.00,1.00,0.99,0.96
Train Recall,0.98,0.99,0.99,0.76
Validation Accuracy,0.85,0.85,0.83,0.81
Validation F1,0.38,0.39,0.36,0.27
Validation Precision,0.26,0.26,0.23,0.18
Validation Recall,0.71,0.73,0.75,0.54


In [37]:
#Random Forest: compare vectorizers with lemmatizing and smote
rfc_results5 = wrapper_compare_vectorizations(X_train.lem_tweet, 
                            y_train, X_val.lem_tweet, y_val, 
                            RandomForestClassifier(random_state=10), 
                            vectorization_list, sampling = 'smote')
rfc_results5

,COUNT_VECTORIZER,TFIDF_VECTORIZER,TFIDF_NGRAM_1_2,TFIDF_NGRAM_2_3
Train Accuracy,0.99,1.00,1.00,0.64
Train F1,0.95,0.98,0.98,0.26
Train Precision,0.95,0.98,0.98,0.15
Train Recall,0.96,0.98,0.98,0.87
Validation Accuracy,0.89,0.94,0.94,0.59
Validation F1,0.37,0.49,0.51,0.18
Validation Precision,0.29,0.51,0.51,0.10
Validation Recall,0.51,0.47,0.50,0.69


### Grid-Searching For Best Fit for Count Vectorizer + Upsampling

In [ ]:
np.random.seed(0)

In [ ]:
X_train_countvect =  count_vect.fit_transform(X_train.lem_tweet)
X_val_countvect =  count_vect.transform(X_val.lem_tweet)
X_test_countvect = count_vect.transform(X_test.lem_tweet)

In [ ]:
rfc = RandomForestClassifier(random_state=10)

parameters = {'n_estimators' : [40, 60, 80, 100],
'max_leaf_nodes' : [200, 400, 600],
'random_state' : [10],
'max_depth': [5, 7, 10, 20],
 'verbose' : [0],
'class_weight': ['balanced', 'balanced_subsample']}
          
rfc_gs = GridSearchCV(rfc, param_grid=parameters, cv = 3)

In [ ]:
rfc_gs.fit(X_train_countvect, y_train)

In [ ]:
rfc_gs.best_params_

In [ ]:
rfc_gs.best_estimator_

In [ ]:
rfc2 = RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=200, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=10, verbose=0, warm_start=False)

In [ ]:
rfc2.fit(X_train_countvect, y_train)

In [ ]:
y_train_predict = rfc2.predict(X_train_countvect)
metrics.f1_score(y_train, y_train_predict)

In [ ]:
y_val_predict = rfc2.predict(X_val_countvect)

In [ ]:
metrics.f1_score(y_val, y_val_predict)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_feature_importances(rfc2):
    n_features = X_val_countvect.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), rfc2.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), countvect.values) 
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")

plot_feature_importances(rfc2)

# Word to Vec

In [ ]:
type(df.tokenized_tweet[0])

In [ ]:
(X_train.tokenized_tweet[0])

In [ ]:
X_train.tokenized_tweet.head()

### X-train pre-processing

In [ ]:
X_train.tokenized_tweet.shape

In [ ]:
X_train_token_list = list(X_train.tokenized_tweet)

In [ ]:
type(X_train_token_list[0])

In [ ]:
X_train_token_sumlist = sum(X_train_token_list,[])

In [ ]:
X_train_unique_tokens = set(X_train_token_sumlist)
print('The unique number of words in the training dataset is: {}'.format(len(X_train_unique_tokens)))

#### X-val pre-processing

In [ ]:
X_val_token_list = list(X_val['tokenized_tweet'])
X_val_token_sumlist = sum(X_val_token_list,[])
X_val_unique_tokens = set(X_val_token_sumlist)

print('The unique number of words in the validation dataset is: {}'.format(len(X_val_unique_tokens)))

#### X-test pre-processing

In [ ]:
X_test_token_list = list(X_test['tokenized_tweet'])
X_test_token_sumlist = sum(X_test_token_list,[])

X_test_unique_tokens = set(X_test_token_sumlist)
print('The unique number of words in the training dataset is: {}'.format(len(X_test_unique_tokens)))

### Training Model

In [ ]:
from time import time
t = time()

w2v_model = gensim.models.Word2Vec(X_train_token_list, sg=1, min_count=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
w2v_model.train(X_train_token_list, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

In [ ]:
w2v_model.save('data/w2v.model')

w2v = gensim.models.Word2Vec.load('data/w2v.model')

In [ ]:
w2v.corpus_count

In [ ]:
w2v_vocab= w2v.wv.vocab

In [ ]:
len(w2v_vocab)

In [ ]:
w2v.wv.vectors.shape

In [ ]:
w2v.wv['trump']

In [ ]:
w2v.wv.most_similar(['trump'])

In [ ]:
w2v.wv.most_similar(['racist'])

In [ ]:
w2v.wv.most_similar(positive=['lazy','black'])

In [ ]:
w2v.wv

In [ ]:
w2v.wv.get_keras_embedding

In [ ]:
w2v_X = w2v.wv.vectors

## Classification with Word2Vec

In [ ]:
sentence = X_train_token_list[1]
sentence

In [ ]:
X_train_w2v[20]

In [ ]:
#create average vector for train and test from model
#returned list of numpy arrays are then stacked 

X_train_w2v_2 = np.concatenate([avg_word_vectors(word, w2v) for word in X_train_token_list])

X_val_w2v_2 = np.concatenate([avg_word_vectors(word, w2v) for word in X_val_token_list])

In [ ]:
X_train_w2v_2[10]

## Classification with Word Embeddings

In [ ]:
smote_w2v_model(X_train_w2v, y_train, X_val_w2v, y_val, LogisticRegression(solver='lbfgs'))

In [ ]:
smote_w2v_model(X_train_w2v_2, y_train, X_val_w2v_2, y_val, LogisticRegression(solver='lbfgs'))

In [ ]:
smote_w2v_model(X_train_w2v_2, y_train, X_val_w2v_2, y_val, RandomForestClassifier())

### Glove

In [ ]:
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove_input_file = 'data/glove.twitter.27B.100d.txt'
# glove_output_file = 'data/glove.txt.word2vec'
# glove2word2vec(glove_input_file, glove_output_file)

In [ ]:
glove_model = KeyedVectors.load_word2vec_format('data/glove.txt.word2vec', binary=False)

In [ ]:
w2v.most_similar('black')

In [ ]:
glove_model.most_similar('black')

In [ ]:
glove_model['love']

In [ ]:
X_train_token_list[0]

In [ ]:
X_train_glove2 = np.empty((20455, 100))
for sentence in X_train_token_list:
    np.append(X_train_glove2, np.mean([glove_model[w] for w in sentence if w in glove_model]
                   or [np.zeros(100)], axis=0))

In [ ]:
X_train_glove = np.concatenate([avg_word_vectors(w, glove_model) for w in X_train_token_list])
X_val_glove = np.concatenate([avg_word_vectors(w, glove_model) for w in X_val_token_list])

In [ ]:
X_train_w2v[10]

In [ ]:
X_train_w2v_2[10]

In [ ]:
X_train_glove[255]

In [ ]:
smote_w2v_model (X_train_glove, y_train, X_val_glove, y_val, LogisticRegression (class_weight='balanced'))

##  Testing Scraped Trump Tweets

In [ ]:
trump_df= pd.read_csv('data/cleaned-trump-tweet.csv')
trump_df.head()

In [ ]:
X_train_countvect =  count_vect.fit_transform(X_train_up.lem_tweet)

In [ ]:
logreg = LogisticRegression(class_weight='balanced')

In [ ]:
logreg.fit(X_train_countvect, y_train_up)

In [ ]:
X_trump = count_vect.transform(trump_df.lem_tweet)

In [ ]:
X_trump = X_trump.toarray()

In [ ]:
X_trump.shape

In [ ]:
y_trump_predict = logreg.predict(X_trump)

In [ ]:
y_trump_predict

In [ ]:
trump_df['predictions'] = y_trump_predict

In [ ]:
y_trump_predict_prob = logreg.predict_proba(X_trump)

In [ ]:
y_trump_predict_prob = pd.DataFrame(y_trump_predict_prob)

In [ ]:
trump_df['predict_probability'] = y_trump_predict_prob[0]

In [ ]:
trump_df = trump_df[['tweet','predictions', 'predict_probability']]


In [ ]:
trump0 = trump_df[trump_df.predictions == 0]

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
trump0.tweet

In [ ]:
trump_df

In [ ]:
trump_df[trump_df.predictions == 0]

In [ ]:
trump1 = trump_df[trump_df.predictions == 1]

In [ ]:
trump1.tweet